# Исследование надежности заемщиков.

## Шаг 1. Обзор данных

In [1]:
#Импортируем библиотеки Pandas
import pandas as pd

In [2]:
#Считываем файл, сохраняем в переменную df, выводим первые 5 строк таблицы
df = pd.read_csv("/datasets/data.csv") 
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
#Узнаем информацию о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Вывод:

Исходя из предоставленной информации, можно сделать вывод о том, что таблица содержит общие данные клиентов банка, такие как: количество детей, трудовой стаж, возраст, уровень образования, семейное положение, пол. Также, финансовые данные: ежемесячный доход, цель получения кредита и информация о задолженностях.

На данном этапе можно выделить такие недостатки, как:
- пропуски в данных 
- дубликаты в данных 
- некорректные типы данных 



## Шаг 2. Предобработка данных
### Заполнение пропусков

In [4]:
#Подсчитываем пропуски
df.isna().sum()


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Судя по результаты, количество пропусков в столбцах 'days_employed'  и 'total_income' одинаково. Проверим в каких строках попадаются пропуски. Следовательно, выясним, появляются ли пропуски в разных строках или они связаны друг с другом.

In [5]:
#Выводим строки с пропусками в выбранных столбцах
passed = df[df['days_employed'].isna() & df['total_income'].isna()]
display(passed.head(10))
print('Количество строк в таблице с пропусками:', passed.shape[0])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Количество строк в таблице с пропусками: 2174


Гипотеза оказалась верна. В двух столбцах, отражающих общий трудовой стаж ('days_employed') и тип занятости ('total_income'), пропуски встречаются и там и там. 
С чем это может быть связано? Вероятно, проблема в выгрузке данных. Однако проблема встречается и у пенсионеров, и у безработных, и у госслужащих и т. п., следовательно, закономерность пропусков не прослеживается. 

Вывод по пропускам:
- Удаление всех пропусков было бы нецелесообразным, тк это слишком большое значение строк от их общего количества.
- Заполнение пустыми строками нелогично, нам нужно узнать 
- Заполнение нулевыми значениями в столбце 'days_employed' может быть разумно, так как этот столбец не повлияет на выводы в исследовании. 
- Заполнить значения в столбце 'total_income' медианными значениями, так как данная информация необходима нам для финального вывода. 


In [6]:
#Выясним медианное значение 
total_income_median = df['total_income'].median()
total_income_median


145017.93753253992

In [7]:
#Заполним пропуски в столбце 'total_income' медианным значением
df['total_income'] = df['total_income'].fillna(value=total_income_median)


In [8]:
#Заполним пропуски в столбце 'days_employed' на медианные значения 
df['days_employed'] = df['days_employed'].median()

In [9]:
#Посмотрим как изменилось количество пропусков в данных после заполнения
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Таким образом, пропуски были устранены. 

### Изменение типов данных.

Заменим вещественный тип данных в столбце 'days_employed'и 'total_income' на целочисленный

In [10]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Отрицательное количество дней трудового стажа в столбце days_employed является аномалией, так как это не является возможным. Это ошибка, которую нужно обработать. Воспользуемся функцией abs(), которая окажет воздействие только на отрицательные значения столбца, оставив положительные значения неимзменными. 

In [11]:
df['days_employed'] = df['days_employed'].abs()

In [12]:
#Убедимся, что отрицательных значений не осталось 
negative_numbers = df[df['days_employed'] < 0].shape[0]
print('Строк с отрицательным значением:', negative_numbers )

Строк с отрицательным значением: 0


### Изменение столбца 'children'

В столбце с количеством детей встречаются отрицательные значения, что является невозможным. Также можно заметить, что существуют 76 записей, которые содержат информацию о том, что у клиента 20 детей. Предположим, что вместо числа 2, по ошибке было записано число 20. 
В итоге, уберем отрицательные значения и заменим 20 на 2. 

In [13]:
#Подсчитаем значения в столбце о детях
df.groupby('children')['children'].count()

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

In [14]:
#Уберем отрицательные значения
df['children'] = df['children'].abs()

In [15]:
#Заменим значения 20 на 2
df['children'] = df['children'].replace(20,2)

In [16]:
#Убедимся, что все исправлено 
df.groupby('children')['children'].count()

children
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Удаление дубликатов.

Для начала избавимся от неявных дубликатов в колонке с образованием клиентов. Можно заметить, что одинаковый уровень образования записан разным регистром в некоторых случаях, исправим это на одинаковый регистр. 

In [17]:
#Ознакомимся с уникальными значениями
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [18]:
#Приведем все значения к нижнему регистру 
df['education'] = df['education'].str.lower()

In [19]:
#Убедимся, что все исправно 
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Далее перейдем к явным дубликатам. Посчитаем их количество. 

In [20]:
df.duplicated().sum()

71

Ознакомимся с этими строками. 

In [21]:
df[df.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,1203,41,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для семьи
3290,0,1203,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
4182,1,1203,34,высшее,0,гражданский брак,1,F,сотрудник,0,145017,свадьба
4851,0,1203,60,среднее,1,гражданский брак,1,F,пенсионер,0,145017,свадьба
5557,0,1203,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,1203,64,среднее,1,женат / замужем,0,F,пенсионер,0,145017,дополнительное образование
21032,0,1203,60,среднее,1,женат / замужем,0,F,пенсионер,0,145017,заняться образованием
21132,0,1203,47,среднее,1,женат / замужем,0,F,сотрудник,0,145017,ремонт жилью
21281,1,1203,30,высшее,0,женат / замужем,0,F,сотрудник,0,145017,покупка коммерческой недвижимости


Изучив данные, можно заметить, что во всех этих строках совпадает заработная плата, что может быть ошибкой. Так как по отношению к общему количеству строк, эти строки составляют маленькое значение, их можно удалить.  

In [22]:
#Удалим данные строки, удаляя старые индексы и формируя новые  
df = df.drop_duplicates().reset_index(drop = True)

In [23]:
#Проверим количество явных дубликатов 
df.duplicated().sum()

0

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма со столбцами:
education_id и education — в первом;
family_status_id и family_status — во втором.

Удалим из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id. 

In [24]:
#Создадим словарь с пронумерованными уникальными значениями, со столбцами education_id и education
education_dict = df[['education_id', 'education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [25]:
family_dict = df[['family_status_id','family_status']]
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
family_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [26]:
df = df.drop(['education', 'family_status'], axis=1)

In [27]:
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,1203,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,1203,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,1203,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,1203,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,1203,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,1203,27,0,1,M,компаньон,0,255763,покупка жилья
6,0,1203,43,0,0,F,компаньон,0,240525,операции с жильем
7,0,1203,50,1,0,M,сотрудник,0,135823,образование
8,2,1203,35,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,1203,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи


### Шаг 2.6. Категоризация дохода.

In [28]:
#Создаем функцию для категоризации 
def total_income_category_func(total_income):
    if total_income <= 30000:
        return 'E'
    if  30001 < total_income < 50000:
        return 'D'
    if 50001 < total_income < 200000:
        return 'C'
    if 200001 < total_income < 1000000:
        return 'B'
    if total_income > 1000001:
        return 'A'
#Создаем новый столбец в общей таблице, результирующий работу функции 
df['total_income_category'] = df['total_income'].apply(total_income_category_func)
df.head(10)
        

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,1203,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,1203,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,1203,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,1203,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,1203,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,1203,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,1203,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,1203,50,1,0,M,сотрудник,0,135823,образование,C
8,2,1203,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,1203,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


### Шаг 2.7. Категоризация целей кредита.

Изучим, какие значения присутствуют в столбце

In [29]:
print(df['purpose'].value_counts().head(30))

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [30]:
#Посмотрим общее количество 
print(len(df['purpose'].unique()))


38


Выделим 4 основные категории взятия кредита:
1. операции с автомобилем,
2. операции с недвижимостью,
3. проведение свадьбы,
4. получение образования


Далее создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут описанные выше категории. 

In [31]:
from pymystem3 import Mystem 

m = Mystem()


def category_purpose_def(row):
    lem_purpose = m.lemmatize(row['purpose'])
    if 'автомобиль' in lem_purpose:
        return 'автомобиль'
    if ('жилье' in lem_purpose) or ('недвижимость' in lem_purpose ) or ('ремонт' in lem_purpose):
        return 'жилье'
    if 'образование' in lem_purpose:
        return 'образование'
    if 'свадьба' in lem_purpose:
        return 'свадьба'

df['purpose_category'] = df.apply(category_purpose_def, axis=1)
df.head(15)


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,1203,42,0,0,F,сотрудник,0,253875,покупка жилья,B,жилье
1,1,1203,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,автомобиль
2,0,1203,33,1,0,M,сотрудник,0,145885,покупка жилья,C,жилье
3,3,1203,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,образование
4,0,1203,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,свадьба
5,0,1203,27,0,1,M,компаньон,0,255763,покупка жилья,B,жилье
6,0,1203,43,0,0,F,компаньон,0,240525,операции с жильем,B,жилье
7,0,1203,50,1,0,M,сотрудник,0,135823,образование,C,образование
8,2,1203,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,свадьба
9,0,1203,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,жилье


### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?


In [32]:
#Получим процент должников в зависимости от количества детей 
pivot_table_children = df.pivot_table(index='children', columns= 'debt', values='days_employed', aggfunc='count')
pivot_table_children['percent'] = pivot_table_children[1]/(pivot_table_children[1] \
                                                                       + pivot_table_children[0]) * 100
pivot_table_children

debt,0,1,percent
children,,,
0,13028.0,1063.0,7.543822
1,4410.0,445.0,9.165808
2,1926.0,202.0,9.492481
3,303.0,27.0,8.181818
4,37.0,4.0,9.756098
5,9.0,NaN,NaN


Следовательно, клиенты без детей имеют самый маленький процент по задолжностям (7%). В отличие от клиентов с 4 детьми, где процент задолжностей равен почти 10%. У клиентов с 3 и 2 детьми ситуация лучше, чем у клиентов с 4 детьми, однако все равно процент задолжностей больше, чем у клиентов без детей. 
В итоге, клиенты не имеющие детей имеют меньше задолженностей, чем клиенты с любым количеством детей. 

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [33]:
#Получим процент должников в зависимости от семейного положения 
pivot_table_family_status = df.pivot_table(index='family_status_id', columns= 'debt', values='days_employed', aggfunc='count')
pivot_table_family_status['percent'] = pivot_table_family_status[1]/(pivot_table_family_status[1] \
                                                                       + pivot_table_family_status[0]) * 100
pivot_table_family_status

debt,0,1,percent
family_status_id,,,
0,11408,931,7.545182
1,3763,388,9.347145
2,896,63,6.569343
3,1110,85,7.112971
4,2536,274,9.750890


Можно сделать вывод, что клиенты, которые не состоят в браке (почти 10%), а также клиенты, состоящие в гражданском браке (9%) чаще всего не выплачивают кредиты в срок. Меньше всего задолжностей у вдовцов/вдов (6%), а также у тех, кто состоит в разводе (7%). У клиентов, состоящих в браке процент должников равен 7%. 

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?


In [34]:
# Получим процент должников относительно дохода клиентов
pivot_table_total_income = df.pivot_table(index='total_income_category', columns= 'debt', values='days_employed', aggfunc='count')
pivot_table_total_income['percent'] = pivot_table_total_income[1]/(pivot_table_total_income[1] \
                                                                       + pivot_table_total_income[0]) * 100
pivot_table_total_income

debt,0,1,percent
total_income_category,,,
A,23,2,8.000000
B,4684,356,7.063492
C,14656,1360,8.491508
D,329,21,6.000000
E,20,2,9.090909


Данные говорят о том, что клиенты категории E, C и А чаще остальных являются должниками. В отличие от категорий B и D, которые чаще платят вовремя. 

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?


In [35]:
#Получим процент должников по каждой цели выдачи кредита
pivot_table_purpose_category = df.pivot_table(index='purpose_category', columns= 'debt', values='days_employed', aggfunc='count')
pivot_table_purpose_category['percent'] = pivot_table_purpose_category[1]/(pivot_table_purpose_category[1] \
                                                                       + pivot_table_purpose_category[0]) * 100
pivot_table_purpose_category 

debt,0,1,percent
purpose_category,,,
автомобиль,3903,403,9.359034
жилье,10029,782,7.233373
образование,3643,370,9.220035
свадьба,2138,186,8.003442


Следовательно, можно сделать вывод, что кредиты по категориям "автомобиль" и "образование" чаще остальных не выплачиваются в срок, а именно больше, чем 9% клиентов несвоевременно проводили платежи. Меньше всего должников по категории "жилье", всего 7%, а также по категории "свадьба" - 8% должников.

## Общий вывод:

Исходя из данных, предоставленных выше, можно сделать общий вывод: семейное положение и количество детей действительно влияют на факт погашения кредита в срок. Клиенты, не состоящие в браке или же состоящие в гражданском браке, в совокуности чаще являются должниками, нежели те клиенты, которые состоят или состояли в браке ранее. Также и клиенты, у которых нет детей, реже являются должниками, чем клиенты, у которых дети есть. 
Также уровень дохода влияет на факт погашения кредита в срок. А именно: клиенты группы "Е", "С" и "А" (перечислены по убыванию по количеству долга) чаще являются должниками, нежели клиенты группы "В" и "D" - которые чаще платят вовремя. 